In [2]:
!python -m pip install semantic-kernel==0.2.3.dev0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.

In [3]:
import semantic_kernel as sk

kernel = sk.Kernel()

* Set OPEN-AI key

In [6]:
from semantic_kernel.ai.open_ai import OpenAITextCompletion

api_key = ""

kernel.config.add_text_backend("dv", OpenAITextCompletion("text-davinci-003", api_key))

In [8]:
kernel.config.add_text_backend(               # We are adding a text backend
    "OpenAI_davinci",                         # The alias we can use in prompt templates' config.json
    OpenAITextCompletion(
        "text-davinci-003",                   # OpenAI Model Name
        ""          # OpenAI API key
    )
)

In [21]:
prompt = """{{$input}}
Translate the content above into telugu and tamil.
"""

summarize = kernel.create_semantic_function(prompt, max_tokens=2000, temperature=0.2, top_p=0.5)

In [22]:
input_text = """
Hi, This is Sravani. I have two friends. They are Dhanasree and Devi Priya.
"""

In [23]:
summary = summarize(input_text)

print(summary)


Telugu: నేను రెండు మిత్రులు ఉన్నాను. అవి ధనస్రీ మరియు దేవీ ప్రియా అనేది.

Tamil: எனக்கு இரு நண்பர்கள் உள்ளன. அவர்கள் தனச்சிரி மற்றும் தேவி ப்ரியா ஆகும்.


In [14]:
sk_prompt = """
{{$input}}

Give me the TLDR in 5 words.
"""

text = """
    1) A robot may not injure a human being or, through inaction,
    allow a human being to come to harm.

    2) A robot must obey orders given it by human beings except where
    such orders would conflict with the First Law.

    3) A robot must protect its own existence as long as such protection
    does not conflict with the First or Second Law.
"""

tldr_function = kernel.create_semantic_function(sk_prompt, max_tokens=200, temperature=0, top_p=0.5)

summary = tldr_function(text)

print(f"Output: {summary}")

Output: 
Robots must not harm humans.


# Chatbot

In [28]:
sk_prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

In [29]:
chat_function = kernel.create_semantic_function(sk_prompt, "ChatBot", max_tokens=2000, temperature=0.7, top_p=0.5)

In [30]:
context = sk.ContextVariables()
context["history"] = ""

In [31]:
context["user_input"] = "Hi, I'm looking for book suggestions"
bot_answer = await chat_function.invoke_with_vars_async(input=context)
print(bot_answer)

 Hi there! What kind of books are you looking for?


In [32]:
context["history"] += f"\nUser: {context['user_input']}\nChatBot: {bot_answer}\n"
print(context["history"])


User: Hi, I'm looking for book suggestions
ChatBot:  Hi there! What kind of books are you looking for?



In [33]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")
    context["user_input"] = input_text

    # Process the user message and get an answer
    answer = await chat_function.invoke_with_vars_async(context)

    # Show the response
    print(f"ChatBot: {answer}")

    # Append the new interaction to the chat history
    context["history"] += f"\nUser: {input_text}\nChatBot: {answer}\n"

In [36]:
await chat("I love to listen some jokes. Can you crack some jokes?")

User: I love to listen some jokes. Can you crack some jokes?
ChatBot:  Sure! Why did the chicken cross the playground? To get to the other slide!


In [38]:
await chat("Some more jokes please")

User: Some more jokes please
ChatBot:  Why did the scarecrow win the Nobel Prize? Because he was outstanding in his field!


In [41]:
await chat("Can you tell jokes in english. But some big jokes instead of one line jokes")

User: Can you tell jokes in english. But some big jokes instead of one line jokes
ChatBot:  Sure! Here's one: A man walks into a bar and orders a drink. The bartender says, "I'm sorry, sir, but you look a lot like my father." The man replies, "That's okay, I'm looking for him too."


In [42]:
await chat("I didnt understand the joke. Can you explain it more")

User: I didnt understand the joke. Can you explain it more
ChatBot:  Sure! The joke is about a man who walks into a bar and orders a drink. The bartender notices that the man looks a lot like his father, so he comments on it. The man replies that he's actually looking for his father, so the joke is that he's not only looking for his father, but he looks like him too.


In [43]:
await chat("I think its a bad joke. Give me some sensible joke")

User: I think its a bad joke. Give me some sensible joke
ChatBot:  Sure! Here's one: What did the fish say when it hit the wall? Dam!
